# Adquisición de Datos
Primero se realizó una generación y búsqueda de datos; pera obtener los mensajes que teminarán siendo integrados en nuestro *dataset* de mensajes; para lo cual, cargamos la informacion haciendo uso de `pandas`.  

Estos datos ya fueron previamente saneados, por lo que no es muy necesario darles un tratemiento programatico de inmediato.

In [1]:
import pandas as pd

df_clasism = pd.read_csv('./datasets/1_clasism.csv')
df_racism = pd.read_csv('./datasets/2_racism.csv')
df_sexism = pd.read_csv('./datasets/3_sexism.csv')
df_other = pd.read_csv('./datasets/4_other.csv')
df_inappropriate = pd.read_csv('./datasets/5_inappropriate_content.csv')
df_none = pd.read_csv('./datasets/6_none_of_the_above.csv')

df_clasism.columns = ['text']
df_racism.columns = ['text']
df_sexism.columns = ['text']
df_other.columns = ['text']
df_inappropriate.columns = ['text']
df_none.columns = ['text']

df_clasism['class'] = 1
df_racism['class'] = 2
df_sexism['class'] = 3
df_other['class'] = 4
df_inappropriate['class'] = 5
df_none['class'] = 6

dataset = pd.concat([df_clasism, df_racism, df_sexism, df_other, df_inappropriate, df_none], ignore_index=True)

dataset.head(), dataset.shape

combined_file_path = './datasets/dataset.csv'
dataset.to_csv(combined_file_path, index=False , mode='w')

# Normalización

Ya que se unificó desde diversas fuentes la información obtenida, podemos hacer un proceso simple de limpieza de datos adicional.

Además, se hace que el texto almacenado esté en letras minusculas, para facilitar el tratamiento de la información y otras modificaciones futuras.  

También, se elimina particularmente una palabra que no brinda informacion del mensaje, sino que hace referencia (de forma anonima) al receptor y/o emisor del mensaje, lo cual no nos es de mucha utilidad, pues no nos permite obtener información valiosa respecto la intención del resto del mensaje.

In [2]:
data = pd.read_csv(combined_file_path)

class_counts = data['class'].value_counts()
class_counts

class
5    489
3    474
2    454
4    449
6    449
1    414
Name: count, dtype: int64

In [3]:
# Que todas las letas sean minusculas
def to_lower(text):
    return text.lower()

data['text'] = data['text'].apply(to_lower)

# Quitar signos de puntuacion
data['text'] = data['text'].str.replace(r"[^a-z0-9\s]", "" , regex=True)

# Guarda el resultado en un nuevo archivo CSV
data.to_csv(combined_file_path, index=False)

data.head()

,text,class
0,public education is in decline luckily my kids...,1
1,it bothers me to have to stand in line like th...,1
2,the poor are only poor because they want to be...,1
3,if you cant afford good wine better not to dri...,1
4,the service in this restaurant is terrible whe...,1


Ya que se tienen los textos en una forma adecuada y agradable para trabajar sobre ellos, podemos empezar a transformalos con un enfoque más apegado al *Procesamiento Natural de Lenguaje*; primero, para tener un procesamiento más optimo, retiramos aquellas palabras que no aportan información relevante, por lo cual retiraremos las **stopwords** encontradas en los textos

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Obtener la lista de stopwords en ingles
stop_words = set(stopwords.words('english'))

# Funcion para tokenizar las palabras
def tokenize(text):
    return word_tokenize(text)

data['tokens'] = data['text'].apply(tokenize)

data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,class,tokens
0,public education is in decline luckily my kids...,1,"[public, education, is, in, decline, luckily, ..."
1,it bothers me to have to stand in line like th...,1,"[it, bothers, me, to, have, to, stand, in, lin..."
2,the poor are only poor because they want to be...,1,"[the, poor, are, only, poor, because, they, wa..."
3,if you cant afford good wine better not to dri...,1,"[if, you, cant, afford, good, wine, better, no..."
4,the service in this restaurant is terrible whe...,1,"[the, service, in, this, restaurant, is, terri..."


Ya que se ha tokenizado exitosamente el texto, se aplica el retirado de stopwords

In [7]:

# Definir una función para eliminar las stopwords de un texto
def remove_stopwords(tokens):
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

# Aplicamos la función a la columna 'tokens' del DataFrame
data['tokens'] = data['tokens'].apply(remove_stopwords)

data.head()

,text,class,tokens
0,public education is in decline luckily my kids...,1,"[public, education, decline, luckily, kids, go..."
1,it bothers me to have to stand in line like th...,1,"[bothers, stand, line, like, rest, plebs, dont..."
2,the poor are only poor because they want to be...,1,"[poor, poor, want, made, hard, work]"
3,if you cant afford good wine better not to dri...,1,"[cant, afford, good, wine, better, drink, anyt..."
4,the service in this restaurant is terrible whe...,1,"[service, restaurant, terrible, manager, deman..."


Ya que aparentemente se ha completado la tokenización y retiro de stepwords exitosamente, procedo a realizar ahora una lematización, de modo que las palabras principales sean "recortadas" a su forma fundamental, y se pueda operar sobre ellas de mejor manera.  
Se prefiere *lematización* sobre *stemming*, pues se desea preservar una raíz adecuada de la palabra; conservando cuanto se pueda, sin alterar los posibles significados o creando accidentalmente palabras inexistentes. Además, que en casos de identificación de motivo e intención, se considera que es más adecuado contar con cuanto contexto y sensibilidad sea adecuado, por lo cual se descarta *stemming*.
### Correccion
Tras comparar los dataframe resultantes de solo efectuar lematizacion, y solo retirar las stopword, se obtuvo el mismo resultado, por lo que, aunque en planteamiento es adecuado primero retirar las stopword, y posteriormente lematizar, a efectos de este proyecto es más útil simplemente retirar las stopword.